In [1]:
import numpy as np
import pandas as pd
import json

In [2]:
%%time
arrest_data = pd.read_csv('data/arrest-data-from-2010-to-present.csv')
crime_data = pd.read_csv('data/crime-data-from-2010-to-present.csv')
income_data = pd.read_csv('data/LAIncome.csv', sep='\t')
moon_data = pd.read_csv('data/full_moon.csv')
race_data = pd.read_csv('data/LARace.csv')
zip_data = pd.read_csv('data/ziplatlon.csv', dtype={'ZIP': 'str'})

CPU times: user 18.8 s, sys: 1.23 s, total: 20 s
Wall time: 20.3 s


In [3]:
arrest_data.columns

Index(['Report ID', 'Arrest Date', 'Time', 'Area ID', 'Area Name',
       'Reporting District', 'Age', 'Sex Code', 'Descent Code',
       'Charge Group Code', 'Charge Group Description', 'Arrest Type Code',
       'Charge', 'Charge Description', 'Address', 'Cross Street', 'Location',
       'Zip Codes', 'Census Tracts', 'Precinct Boundaries',
       'LA Specific Plans', 'Council Districts',
       'Neighborhood Councils (Certified)'],
      dtype='object')

In [4]:
income_data.head()

,Zip,Community,Amount
0,90001,"Los Angeles (South Los Angeles), Florence-Graham","$43,360"
1,90002,"Los Angeles (Southeast Los Angeles, Watts)","$37,285"
2,90003,"Los Angeles (South Los Angeles, Southeast Los ...","$40,598"
3,90004,"Los Angeles (Hancock Park, Rampart Village, Vi...","$49,675"
4,90005,"Los Angeles (Hancock Park, Koreatown, Wilshire...","$38,491"


In [5]:
moon_data.head()

,Day,Date,Time
0,Monday,15 January 1900,08:07:30 pm
1,Wednesday,14 February 1900,02:50:12 pm
2,Friday,16 March 1900,09:11:48 am
3,Sunday,15 April 1900,02:02:06 am
4,Monday,14 May 1900,04:36:36 pm


### Dealing with location

In [25]:
eval(arrest_data['Location'][0])

{'latitude': '33.992',
 'human_address': '{"address": "", "city": "", "state": "", "zip": ""}',
 'longitude': '-118.4201'}

In [7]:
race_data['Zip Code'].nunique()

110

In [29]:
def convert_lat_lon(x):
    return (eval(x)['latitude'], eval(x)['longitude']) 
convert_lat_lon = np.vectorize(convert_lat_lon)

In [30]:
arrest_data['Lat'], arrest_data['Lon'] = convert_lat_lon(arrest_data['Location'])

In [42]:
zip_data = zip_data[zip_data['ZIP'].apply(lambda x: x[:2]=='90')]

In [49]:
sample_lat, sample_lon = arrest_data['Lat'][0], arrest_data['Lon'][0] 

In [84]:
temp = pd.DataFrame( { 'ZIP':zip_data['ZIP'], 'DifLat': zip_data['LAT'] - float(sample_lat) , 'DifLon': zip_data['LNG'] - float(sample_lon)})
temp['ToMin'] = temp['DifLat']**2 + temp['DifLon']**2
temp

,ZIP,DifLat,DifLon,ToMin
30021,90001,-0.017973,0.170591,0.029424
30022,90002,-0.042901,0.173363,0.031895
30023,90003,-0.027869,0.147317,0.022479
30024,90004,0.084198,0.109378,0.019053
30025,90005,0.067163,0.113208,0.017327
...,...,...,...,...
30187,90813,-0.209741,0.223307,0.093857
30188,90814,-0.220384,0.276469,0.125004
30189,90815,-0.197652,0.303709,0.131305
30190,90822,-0.213564,0.301452,0.136483


In [97]:
temp[temp['ToMin']==temp['ToMin'].min()]['ZIP'].iloc[0]

'90066'

In [88]:
sample_lat, sample_lon

('33.992', '-118.4201')

In [101]:
def calc_zip(lat, lon):
    temp = pd.DataFrame( { 'ZIP':zip_data['ZIP'], 'DifLat': zip_data['LAT'] - float(lat) , 'DifLon': zip_data['LNG'] - float(lon)})
    temp['ToMin'] = temp['DifLat']**2 + temp['DifLon']**2
    return temp[temp['ToMin']==temp['ToMin'].min()]['ZIP'].iloc[0]
calc_zip = np.vectorize(calc_zip)

In [103]:
%%time
arrest_data['ZipCode'] = calc_zip(arrest_data['Lat'], arrest_data['Lon'])

CPU times: user 1h 2min 55s, sys: 29.9 s, total: 1h 3min 25s
Wall time: 1h 3min 48s


In [122]:
arrest_data.to_csv('arrest_with_correct_zipcode.csv', index=False)

In [123]:
%%time
pd.read_csv('arrest_with_correct_zipcode.csv')

CPU times: user 4.39 s, sys: 184 ms, total: 4.57 s
Wall time: 4.57 s


,Report ID,Arrest Date,Time,Area ID,Area Name,Reporting District,Age,Sex Code,Descent Code,Charge Group Code,...,Location,Zip Codes,Census Tracts,Precinct Boundaries,LA Specific Plans,Council Districts,Neighborhood Councils (Certified),Lat,Lon,ZipCode
0,5666847,2019-06-22T00:00:00.000,1630.0,14,Pacific,1457,44,M,W,24.0,...,"{'latitude': '33.992', 'human_address': '{""add...",24031.0,918.0,1137.0,10.0,10.0,85.0,33.9920,-118.4201,90066
1,5666688,2019-06-22T00:00:00.000,1010.0,10,West Valley,1061,8,M,O,NaN,...,"{'latitude': '34.1687', 'human_address': '{""ad...",19339.0,321.0,1494.0,NaN,4.0,10.0,34.1687,-118.5579,90272
2,5666570,2019-06-22T00:00:00.000,400.0,15,N Hollywood,1543,31,F,O,22.0,...,"{'latitude': '34.1649', 'human_address': '{""ad...",8890.0,205.0,1332.0,17.0,5.0,39.0,34.1649,-118.3965,90046
3,5666529,2019-06-22T00:00:00.000,302.0,17,Devonshire,1738,23,F,W,22.0,...,"{'latitude': '34.2692', 'human_address': '{""ad...",19329.0,69.0,388.0,NaN,2.0,78.0,34.2692,-118.4789,90077
4,5666742,2019-06-22T00:00:00.000,1240.0,14,Pacific,1472,28,M,W,8.0,...,"{'latitude': '33.9609', 'human_address': '{""ad...",25075.0,937.0,241.0,10.0,10.0,16.0,33.9609,-118.4504,90292
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1276155,100504416,2010-01-01T00:00:00.000,1430.0,5,Harbor,521,17,M,H,24.0,...,"{'latitude': '33.7406', 'human_address': '{""ad...",3342.0,975.0,1205.0,NaN,15.0,36.0,33.7406,-118.2923,90731
1276156,101104731,2010-01-01T00:00:00.000,2215.0,11,Northeast,1118,12,M,H,24.0,...,"{'latitude': '34.1101', 'human_address': '{""ad...",23673.0,370.0,477.0,28.0,11.0,93.0,34.1101,-118.1918,90042
1276157,101104211,2010-01-01T00:00:00.000,1310.0,11,Northeast,1128,52,M,H,18.0,...,"{'latitude': '34.1148', 'human_address': '{""ad...",23673.0,359.0,575.0,NaN,9.0,93.0,34.1148,-118.1826,90042
1276158,2179817,2010-01-01T00:00:00.000,319.0,14,Pacific,1408,24,M,H,22.0,...,"{'latitude': '34.0301', 'human_address': '{""ad...",23451.0,872.0,1124.0,9.0,6.0,75.0,34.0301,-118.4029,90034


In [118]:
crime_data['Lat'], crime_data['Lon'] = convert_lat_lon(crime_data['Location '])

In [119]:
%%time
crime_data['ZipCode'] = calc_zip(crime_data['Lat'], crime_data['Lon'])

CPU times: user 1h 8min 57s, sys: 689 ms, total: 1h 8min 58s
Wall time: 1h 8min 57s


In [120]:
crime_data.to_csv('crime_with_correct_zipcode.csv', index=False)

In [121]:
%%time
pd.read_csv('crime_with_correct_zipcode.csv')

CPU times: user 8.14 s, sys: 464 ms, total: 8.6 s
Wall time: 8.6 s


,DR Number,Date Reported,Date Occurred,Time Occurred,Area ID,Area Name,Reporting District,Crime Code,Crime Code Description,MO Codes,...,Crime Code 1,Crime Code 2,Crime Code 3,Crime Code 4,Address,Cross Street,Location,Lat,Lon,ZipCode
0,102005556,2010-01-25T00:00:00,2010-01-22T00:00:00,2300,20,Olympic,2071,510,VEHICLE - STOLEN,NaN,...,510.0,NaN,NaN,NaN,VAN NESS,15TH,"{'latitude': '34.0454', 'needs_recoding': Fals...",34.0454,-118.3157,90005
1,101822289,2010-11-11T00:00:00,2010-11-10T00:00:00,1800,18,Southeast,1803,510,VEHICLE - STOLEN,NaN,...,510.0,NaN,NaN,NaN,88TH,WALL,"{'latitude': '33.9572', 'needs_recoding': Fals...",33.9572,-118.2717,90003
2,101105609,2010-01-28T00:00:00,2010-01-27T00:00:00,2230,11,Northeast,1125,510,VEHICLE - STOLEN,NaN,...,510.0,NaN,NaN,NaN,YORK,AVENUE 51,"{'latitude': '34.1211', 'needs_recoding': Fals...",34.1211,-118.2048,90042
3,101620051,2010-11-11T00:00:00,2010-11-07T00:00:00,1600,16,Foothill,1641,510,VEHICLE - STOLEN,NaN,...,510.0,NaN,NaN,NaN,EL DORADO,TRUESDALE,"{'latitude': '34.241', 'needs_recoding': False...",34.2410,-118.3987,90068
4,101910498,2010-04-07T00:00:00,2010-04-07T00:00:00,1600,19,Mission,1902,510,VEHICLE - STOLEN,NaN,...,510.0,NaN,NaN,NaN,GLENOAKS,DRELL,"{'latitude': '34.3147', 'needs_recoding': Fals...",34.3147,-118.4589,90077
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1993254,192111884,6242019-01-01T00:00:00,2019-06-21T00:00:00,1205,21,Topanga,2143,442,SHOPLIFTING - PETTY THEFT ($950 & UNDER),0325,...,442.0,NaN,NaN,NaN,6600 FALLBROOK AV,NaN,"{'latitude': '34.1883', 'needs_recoding': Fals...",34.1883,-118.6274,90290
1993255,199905748,1152019-01-01T00:00:00,2019-01-15T00:00:00,1940,18,Southeast,1829,910,KIDNAPPING,2000 1243 0416 0515 1402,...,910.0,NaN,NaN,NaN,2000 E 101ST ST,NaN,"{'latitude': '33.9449', 'needs_recoding': Fals...",33.9449,-118.2367,90002
1993256,199907720,2022019-01-01T00:00:00,2019-02-01T00:00:00,2126,11,Northeast,1126,946,OTHER MISCELLANEOUS CRIME,1407 1300 1309 0906,...,946.0,998.0,NaN,NaN,CORINGA,N AVENUE 53,"{'latitude': '34.1244', 'needs_recoding': Fals...",34.1244,-118.1985,90042
1993257,199916823,4262019-01-01T00:00:00,2019-02-05T00:00:00,1600,17,Devonshire,1794,668,"EMBEZZLEMENT, GRAND THEFT ($950.01 & OVER)",NaN,...,668.0,NaN,NaN,NaN,8400 RESEDA BL,NaN,"{'latitude': '34.2227', 'needs_recoding': Fals...",34.2227,-118.5361,90272
